In [12]:
from bs4 import BeautifulSoup
import requests
import csv
import re
import pandas as pd

In [14]:
df_house = pd.read_csv("beijinghouse_detail.csv")
df_house.head()

,heat_con,layout,location,neighborhood,type_layout,area,building,decoration,direction,heat,name,price,subway,url,visiting,zone,zoneinfo
0,集中供暖,整租,广安门,西城,房屋户型：1室1厅1卫 整租,40平米,车站西街17号院,NaN,南,NaN,车站西街17号院1居室,5530,NaN,https://bj.lianjia.com/zufang/BJ0004868491.html,NaN,广安门租房/高楼层(共18层)/1993年建塔楼,1室1厅
1,集中供暖,整租,洋桥,丰台,房屋户型：1室1厅1卫 整租,45平米,海户西里,原味,南,集中供暖,海户西里1居室,5960,近地铁,https://bj.lianjia.com/zufang/BJ0004872617.html,随时看房,洋桥租房/低楼层(共18层)/1993年建塔楼,1室1厅
2,集中供暖,整租,广安门,西城,房屋户型：1室1厅1卫 整租,46平米,车站西街17号院,NaN,南,NaN,车站西街17号院1居室,5430,NaN,https://bj.lianjia.com/zufang/BJ0004872565.html,NaN,广安门租房/高楼层(共6层)/1992年建板楼,1室1厅
3,集中供暖,整租,马甸,西城,房屋户型：1室1厅1卫 整租,60平米,阳光丽景,原味,北,集中供暖,阳光丽景1居室,8930,近地铁,https://bj.lianjia.com/zufang/BJ0004879242.html,随时看房,马甸租房/低楼层(共16层)/2003年建板塔结合,1室1厅
4,自供暖,合租,回龙观,昌平,房屋户型：3室1厅1卫 合租,15平米,龙腾苑三区,自如友家4.0拿铁,南,自供暖,龙腾苑三区3居室 01卧,3690,近地铁,https://bj.lianjia.com/zufang/BJ0004846189.html,随时看房,回龙观租房/低楼层(共6层)/2002年建板楼,3室1厅


In [23]:
def getlocation(row):#调用百度API查询位置
    bdurl='http://api.map.baidu.com/geocoder/v2/?address='
    output='json'
    ak='1e2TPBxeffGukrPj4twVv4inGLafAqPr'#输入你刚才申请的密匙
    callback='showLocation'
    uri=bdurl+row['building']+'&output=t'+output+'&ak='+ak+'&callback='+callback
    res=requests.get(uri)
    s=BeautifulSoup(res.text)
    lng=s.find('lng')
    lat=s.find('lat')
    if lng:
        return pd.Series ({
            'longitude':lng.get_text(),
            'latitude':lat.get_text()
            
        })


In [24]:
df_lat = df_house.apply(getlocation, axis=1).join(df_house)
df_lat

/Users/siruizhu/.pyenv/versions/3.6.5/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/siruizhu/.pyenv/versions/3.6.5/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


,longitude,latitude,heat_con,layout,location,neighborhood,type_layout,area,building,decoration,direction,heat,name,price,subway,url,visiting,zone,zoneinfo
0,116.346931172,39.8845892012,集中供暖,整租,广安门,西城,房屋户型：1室1厅1卫 整租,40平米,车站西街17号院,NaN,南,NaN,车站西街17号院1居室,5530,NaN,https://bj.lianjia.com/zufang/BJ0004868491.html,NaN,广安门租房/高楼层(共18层)/1993年建塔楼,1室1厅
1,116.396765204,39.8573600478,集中供暖,整租,洋桥,丰台,房屋户型：1室1厅1卫 整租,45平米,海户西里,原味,南,集中供暖,海户西里1居室,5960,近地铁,https://bj.lianjia.com/zufang/BJ0004872617.html,随时看房,洋桥租房/低楼层(共18层)/1993年建塔楼,1室1厅
2,116.346931172,39.8845892012,集中供暖,整租,广安门,西城,房屋户型：1室1厅1卫 整租,46平米,车站西街17号院,NaN,南,NaN,车站西街17号院1居室,5430,NaN,https://bj.lianjia.com/zufang/BJ0004872565.html,NaN,广安门租房/高楼层(共6层)/1992年建板楼,1室1厅
3,116.931683175,38.7140609627,集中供暖,整租,马甸,西城,房屋户型：1室1厅1卫 整租,60平米,阳光丽景,原味,北,集中供暖,阳光丽景1居室,8930,近地铁,https://bj.lianjia.com/zufang/BJ0004879242.html,随时看房,马甸租房/低楼层(共16层)/2003年建板塔结合,1室1厅
4,116.338076976,40.0827958415,自供暖,合租,回龙观,昌平,房屋户型：3室1厅1卫 合租,15平米,龙腾苑三区,自如友家4.0拿铁,南,自供暖,龙腾苑三区3居室 01卧,3690,近地铁,https://bj.lianjia.com/zufang/BJ0004846189.html,随时看房,回龙观租房/低楼层(共6层)/2002年建板楼,3室1厅
5,101.730143411,26.5886092059,暂无数据,合租,白石桥,海淀,房屋户型：3室1厅1卫 合租,12平米,钢铁研究院,NaN,南,NaN,钢铁研究院3居室 02卧,4790,NaN,https://bj.lianjia.com/zufang/BJ0004883867.html,NaN,白石桥租房/中楼层(共16层)/1994年建塔楼,3室1厅
6,116.396069667,39.8570391911,集中供暖,合租,洋桥,丰台,房屋户型：3室1厅1卫 合租,7平米,海户西里33号院,NaN,北,NaN,海户西里33号院3居室 02卧,2390,NaN,https://bj.lianjia.com/zufang/BJ0004886918.html,NaN,洋桥租房/中楼层(共18层)/1996年建塔楼,3室1厅
7,116.392411099,39.862531873,集中供暖,整租,洋桥,丰台,房屋户型：2室1厅1卫 整租,56平米,马家堡东里,NaN,东,NaN,马家堡东里2居室,6330,NaN,https://bj.lianjia.com/zufang/BJ0004803941.html,NaN,洋桥租房/高楼层(共6层)/1986年建板楼,2室1厅
8,116.338076976,40.0827958415,自供暖,合租,回龙观,昌平,房屋户型：3室1厅1卫 合租,15平米,龙腾苑三区,自如友家4.0拿铁,南,自供暖,龙腾苑三区3居室 01卧,3160,近地铁,https://bj.lianjia.com/zufang/BJ0004803331.html,随时看房,回龙观租房/低楼层(共6层)/2002年建板楼,3室1厅
9,116.375517586,40.0443789445,集中供暖,合租,清河,海淀,房屋户型：3室1厅1卫 合租,10平米,宝盛北里,NaN,南,NaN,宝盛北里3居室 03卧,3760,NaN,https://bj.lianjia.com/zufang/BJ0004806559.html,NaN,清河租房/高楼层(共6层)/2009年建板楼,3室1厅


In [27]:
df_house_total = pd.DataFrame(df_lat)

In [29]:
df_house_total.to_csv("df_beijinghouse_geo.csv", index=False)